In [1]:
import argparse
import select
import socket
import sys
from Crypto.Cipher import AES as AES
from Crypto.Hash import HMAC as HMAC
from Crypto.Hash import SHA256 as SHA256
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes as randb

In [2]:
message = "test 553 and that's it"
K1 = "adfjkiwdhfiuh"
K2 = "askdijhasoidaasdas"

In [126]:
# iv + E_k1(len(m)) + HMAC_k2(iv + E_k1(len(m))) + E_k1(m) + HMAC_k2(E_k1(m))

IV = randb(16)

# Create a 256-bit key for AES encryption/decryption by using the SHA256 hash of the K1 key.
K1_256 = SHA256.new(K1.encode()).digest()
# The encode() method turns the string into bytes, needed for the SHA256.new() method.
# The digest() method returns the final 256 bit hash in binary.

# Create an HMAC instance by using the SHA256 hash of the K2 key.
K2_HMAC = HMAC.new(K2.encode(), digestmod=SHA256)
# Repeat for the message
K2_HMAC_message = HMAC.new(K2.encode(), digestmod=SHA256)

# Create an AES instance for encryption/decryption by using the SHA256 hash of the K1 key.
K1_cipher = AES.new(key=K1_256, mode=AES.MODE_CBC, iv=IV)

# Create a binary object which is the encryption of the length of the message (padded)
K1_length = K1_cipher.encrypt(pad(len(message).to_bytes(), AES.block_size))
# Create a binary object which is the encryption of the message (padded)
K1_message = K1_cipher.encrypt(pad(message.encode('utf-8'), AES.block_size))

HMAC_IV_length = K2_HMAC.update(IV + K1_length).digest()
HMAC_message = K2_HMAC_message.update(K1_message).digest()
enc_message = IV + K1_length + HMAC_IV_length + K1_message + HMAC_message

print("\nmessagelength")
print(len(message).to_bytes())

print("\nIV")
print(IV)

print("\nK1_length")
print(K1_length)

print("\nHMAC_IV_length")
print(HMAC_IV_length)

print("\nK1_message")
print(K1_message)

print("\nHMAC_message")
print(HMAC_message)

print("\nFull message")
print(enc_message)


messagelength
b'\x16'

IV
b'U<\xbcC\xd9\x91\xfe\x8a\xb3\x1a\xe5\xd2\x12\x13i\xa1'

K1_length
b'\x9a\xdb)\xa7\x7fMZ\xa3\xe7R\x92\x10BA\xbd\xec'

HMAC_IV_length
b"\x89\x87o\x1b\xd1a\xb8i\x8a\xb8\x8a\x0c\xf7\xbcxA\xcbI\x0b'\x0c\x05\xe6g6\x88\xb1\xaeF\x11R\xd5"

K1_message
b'5\xafuA\x15$y.n\xferP b6\x80\x19Ez\x13\x81\xd1\xb0\x98f\xbe\xfd|\xe5\xbc\x91I'

HMAC_message
b'\xf6m\xb6\xedc\x90.\x08$\xa7}ra\xe9\xdb\xa4\x06\xf0\xb5\x80\x00z\xb0\xd2u\x01\xa7\xc1B\xcf\xfb\x1e'

Full message
b"U<\xbcC\xd9\x91\xfe\x8a\xb3\x1a\xe5\xd2\x12\x13i\xa1\x9a\xdb)\xa7\x7fMZ\xa3\xe7R\x92\x10BA\xbd\xec\x89\x87o\x1b\xd1a\xb8i\x8a\xb8\x8a\x0c\xf7\xbcxA\xcbI\x0b'\x0c\x05\xe6g6\x88\xb1\xaeF\x11R\xd55\xafuA\x15$y.n\xferP b6\x80\x19Ez\x13\x81\xd1\xb0\x98f\xbe\xfd|\xe5\xbc\x91I\xf6m\xb6\xedc\x90.\x08$\xa7}ra\xe9\xdb\xa4\x06\xf0\xb5\x80\x00z\xb0\xd2u\x01\xa7\xc1B\xcf\xfb\x1e"


In [147]:
IVdec = enc_message[:16]

print("IVdec:")
print(IVdec)

# iv + E_k1(len(m))+HMAC_k2(iv + E_k1(len(m))) + E_k1(m) + HMAC_k2(E_k1(m))

# Extract K1_length
K1_lengthdec = enc_message[16:32]  # K1_length is 32 bytes (256 bits)

print("\nK1_lengthdec")
print(K1_lengthdec)

# Extract the HMAC of IV + K1_length
K2_HMAC_IV_K1_lengthdec = enc_message[32:64]  # The HMAC used SHA256 so it is 32 bytes (256 bits)

# Create a 256-bit key for HMAC computation by using the SHA256 hash of the K2 key.
K2_HMACdec = HMAC.new(K2.encode(), digestmod=SHA256)
HMAC_IV_lengthdec = K2_HMACdec.update(IV + K1_lengthdec).digest()
print("\nHMAC_IV_lengthdec:")
print(HMAC_IV_lengthdec)

# print(K2_HMAC.update(IV + K1_length).digest() )
# # K2_HMACdec.update(IVdec + K1_lengthdec)

print("\nK2_HMAC_IV_K1_lengthdec")
print(K2_HMAC_IV_K1_lengthdec)
# Validate that IV and length are unchanged
if K2_HMACdec.digest() != K2_HMAC_IV_K1_lengthdec:
    print("ERROR: HMAC verification failed")
    sock.shutdown(socket.SHUT_RDWR)
    sock.close()
    sys.exit(0)

K1_256dec = SHA256.new(K1.encode()).digest()
K1_cipherdec = AES.new(key=K1_256dec, mode=AES.MODE_CBC, iv=IVdec)

lengthdec = int.from_bytes(unpad(K1_cipherdec.decrypt(K1_lengthdec), AES.block_size))
print("\nlengthdec")
print(lengthdec)

K1_messagedec = enc_message[64:-32]
HMAC_messagedec = enc_message[-32:]
print(K1_messagedec)
print(HMAC_messagedec)

K2_HMAC_message_inst = HMAC.new(K2.encode(), digestmod=SHA256)
K2_HMAC_messagedec = K2_HMAC_message_inst.update(K1_messagedec).digest()
print("\nHMAC_messagedec:")
print(K2_HMAC_messagedec)
if K2_HMAC_messagedec != HMAC_messagedec:
    print("ERROR: HMAC verification failed")
    sock.shutdown(socket.SHUT_RDWR)
    sock.close()
    sys.exit(0)


messagedec = unpad(K1_cipherdec.decrypt(K1_messagedec), AES.block_size).decode('utf-8')
print(messagedec)

IVdec:
b'U<\xbcC\xd9\x91\xfe\x8a\xb3\x1a\xe5\xd2\x12\x13i\xa1'

K1_lengthdec
b'\x9a\xdb)\xa7\x7fMZ\xa3\xe7R\x92\x10BA\xbd\xec'

HMAC_IV_lengthdec:
b"\x89\x87o\x1b\xd1a\xb8i\x8a\xb8\x8a\x0c\xf7\xbcxA\xcbI\x0b'\x0c\x05\xe6g6\x88\xb1\xaeF\x11R\xd5"

K2_HMAC_IV_K1_lengthdec
b"\x89\x87o\x1b\xd1a\xb8i\x8a\xb8\x8a\x0c\xf7\xbcxA\xcbI\x0b'\x0c\x05\xe6g6\x88\xb1\xaeF\x11R\xd5"

lengthdec
22
b'5\xafuA\x15$y.n\xferP b6\x80\x19Ez\x13\x81\xd1\xb0\x98f\xbe\xfd|\xe5\xbc\x91I'
b'\xf6m\xb6\xedc\x90.\x08$\xa7}ra\xe9\xdb\xa4\x06\xf0\xb5\x80\x00z\xb0\xd2u\x01\xa7\xc1B\xcf\xfb\x1e'

HMAC_messagedec:
b'\xf6m\xb6\xedc\x90.\x08$\xa7}ra\xe9\xdb\xa4\x06\xf0\xb5\x80\x00z\xb0\xd2u\x01\xa7\xc1B\xcf\xfb\x1e'
test 553 and that's it


In [19]:
IV = randb(16)

# Create a 256-bit key for AES encryption/decryption by using the SHA256 hash of the K1 key.
K1_256 = SHA256.new(K1.encode()).digest()
# The encode() method turns the string into bytes, needed for the SHA256.new() method.
# The digest() method returns the final 256 bit hash in binary.

# Create an HMAC instance by using the SHA256 hash of the K2 key.
K2_HMAC = HMAC.new(K2.encode(), digestmod=SHA256)
# Repeat for the message
K2_HMAC_message = HMAC.new(K2.encode(), digestmod=SHA256)

# Create an AES instance for encryption/decryption by using the SHA256 hash of the K1 key.
K1_cipher = AES.new(key=K1_256, mode=AES.MODE_CBC, iv=IV)

# Create a binary object which is the encryption of the length of the message (padded)
K1_length = K1_cipher.encrypt(pad(len(message).to_bytes(15,'little'), AES.block_size))
print(len(K1_length))
# Create a binary object which is the encryption of the message (padded)
K1_message = K1_cipher.encrypt(pad(message.encode('utf-8'), AES.block_size))

HMAC_IV_length = K2_HMAC.update(IV + K1_length).digest()
HMAC_message = K2_HMAC_message.update(K1_message).digest()
enc_message = IV + K1_length + HMAC_IV_length + K1_message + HMAC_message

print(enc_message)

16
b'\r\xf6F\x19o\x11\x01V\x08\xc1`(}\t\x14\x85e\xeep\xe5\xcdJ!\x84\\\xacHW\xc6\xc9\xb4=\xadQ\x88\xa1\xc9t\xb2W\xa2\x1bn\xf7\xe43\xefr\xff\x06[C6\xc5/\xa8\xd4\n\xab\xe2\xe0\x8c\x1b]\xe7\xfcf\xdcRv\x1a\x8a\x7f\x00\x08\xecwN\x18j_\x05\xe4p\x89\x83\x13\xdc|G\x9d8\xa4\xcc\xf4\xcf\x00\x8c\xe4\x159x\xbb\xe8\x15\xc5q\x89i$\xf9\xf29\xd6\x88\x9a\xed\x0f%<a W\xf4*|X\x0c'


In [27]:
IV = enc_message[:16]

# Extract K1_length
K1_length = enc_message[16:32]  # K1_length is 16 bytes (128 bits)

# Extract the HMAC of IV + K1_length
K2_HMAC_IV_K1_length = enc_message[32:64]  # The HMAC used SHA256 so it is 32 bytes (256 bits)

# Create a 256-bit key for HMAC computation by using the SHA256 hash of the K2 key.
K2_HMAC = HMAC.new(K2.encode(), digestmod=SHA256)
HMAC_IV_length = K2_HMAC.update(IV + K1_length).digest()

# Validate that IV and length are unchanged
if K2_HMAC.digest() != K2_HMAC_IV_K1_length:
    print("ERROR: HMAC verification failed1st")
    # sock.shutdown(socket.SHUT_RDWR)
    # sock.close()
    # sys.exit(0)

K1_256 = SHA256.new(K1.encode()).digest()
K1_cipher = AES.new(key=K1_256, mode=AES.MODE_CBC, iv=IV)

length = int.from_bytes(unpad(K1_cipher.decrypt(K1_length), AES.block_size),'little')

K1_message = enc_message[64:96]
HMAC_message = enc_message[-32:]

print(64+length)
print(len(enc_message)-32)
K2_HMAC_message_inst = HMAC.new(K2.encode(), digestmod=SHA256)
K2_HMAC_message = K2_HMAC_message_inst.update(K1_message).digest()

if K2_HMAC_message != HMAC_message:
    print("ERROR: HMAC verification failed")
    # sock.shutdown(socket.SHUT_RDWR)
    # sock.close()
    # sys.exit(0)

message = unpad(K1_cipher.decrypt(K1_message), AES.block_size).decode('utf-8')
print(message)

86
96
test 553 and that's it
